pandas join

In [1]:
import pandas as pd

In [2]:
d1 = {'customer_id':[1, 2, 3, 4, 5, 6],
       'product' : ['Oven', 'Oven', 'Oven', 'Television', 'Television', 'Television']}
df1 = pd.DataFrame(d1)
d2 = {'customer_id':[2, 4, 6, 8],
      'state':['California', 'California', 'Texas', 'Hell']}
df2 = pd.DataFrame(d2)

In [3]:
df1

,customer_id,product
0,1,Oven
1,2,Oven
2,3,Oven
3,4,Television
4,5,Television
5,6,Television


In [4]:
df2

,customer_id,state
0,2,California
1,4,California
2,6,Texas
3,8,Hell


In [5]:
# Inner Join : 교집합
inner_join_result = pd.merge(df1, df2, on='customer_id', how='inner')
inner_join_result

,customer_id,product,state
0,2,Oven,California
1,4,Television,California
2,6,Television,Texas


In [9]:
inner_join_result_01 = df1.merge(df2, on='customer_id', how='inner')
inner_join_result_01

,customer_id,product,state
0,2,Oven,California
1,4,Television,California
2,6,Television,Texas


In [6]:
# Left Outer Join : 왼쪽 값 + 교집합
left_outer_join_result = df1.merge(df2, on='customer_id', how='left')
left_outer_join_result

,customer_id,product,state
0,1,Oven,NaN
1,2,Oven,California
2,3,Oven,NaN
3,4,Television,California
4,5,Television,NaN
5,6,Television,Texas


In [7]:
# Right Outer Join : 교집합 + 오른쪽 값
right_outer_join_result = df1.merge(df2, on='customer_id', how='right')
print(right_outer_join_result)
print('======================================')
left_outer_join_result_01 = df2.merge(df1, on='customer_id', how='left')
print(left_outer_join_result_01)

   customer_id     product       state
0            2        Oven  California
1            4  Television  California
2            6  Television       Texas
3            8         NaN        Hell
   customer_id       state     product
0            2  California        Oven
1            4  California  Television
2            6       Texas  Television
3            8        Hell         NaN


In [8]:
# Full Outer Join : 합집합
full_outer_join_result = pd.merge(df1, df2, on='customer_id', how='outer')
full_outer_join_result

,customer_id,product,state
0,1,Oven,NaN
1,2,Oven,California
2,3,Oven,NaN
3,4,Television,California
4,5,Television,NaN
5,6,Television,Texas
6,8,NaN,Hell


pandas agg()

In [10]:
cust_dict = {
    'customer_name':['Alice', 'Tom', 'James', 'Yerin', 'Min', 'Katherine', 'John', 'Park', 'Maria', 'Derik', 'Jin'],
    'cust_country':['US', 'GB', 'US', 'KOR', 'KOR', 'GB', 'US', 'KOR', 'US', 'GB', 'KOR'],
    'gender':['F', 'M', 'M', 'F', 'M', 'F', 'M', 'M', 'F', 'M', 'F'],
    'grade':[3, 1, 2, 5, 4, 3, 5, 7, 2, 4, 5],
    'age':[25, 34, 26, 33, 67, 29, 54, 21, 77, 29, 16]
}

In [11]:
customer = pd.DataFrame(cust_dict)
customer.head()

,customer_name,cust_country,gender,grade,age
0,Alice,US,F,3,25
1,Tom,GB,M,1,34
2,James,US,M,2,26
3,Yerin,KOR,F,5,33
4,Min,KOR,M,4,67


- SQL Group by  
SELECT cust_country, sum(grade) sum_grade, max(grade) max_grade, avg(age) avg_age, 
max(age) max_age FROM customer group by cust_country  

- Pandas Group by  
group key 컬럼으로 DataFrameGroupBy를 생성

In [12]:
cust_group = customer.groupby('cust_country')
type(cust_group)

pandas.core.groupby.generic.DataFrameGroupBy

In [33]:
customer.groupby('cust_country')['grade'].sum()

cust_country
GB      8
KOR    21
US     12
Name: grade, dtype: int64

In [13]:
cust_group.head()

,customer_name,cust_country,gender,grade,age
0,Alice,US,F,3,25
1,Tom,GB,M,1,34
2,James,US,M,2,26
3,Yerin,KOR,F,5,33
4,Min,KOR,M,4,67
5,Katherine,GB,F,3,29
6,John,US,M,5,54
7,Park,KOR,M,7,21
8,Maria,US,F,2,77
9,Derik,GB,M,4,29


개별 aggregation 컬럼 및 aggregation 연산별로 API 호출

In [15]:
cust_agg = pd.DataFrame()
cust_agg['sum_grade'] = cust_group['grade'].sum()
cust_agg['max_grade'] = cust_group['grade'].max()
cust_agg.head()

,sum_grade,max_grade
cust_country,,
GB,8,4
KOR,21,7
US,12,5


In [16]:
# group by 키 컬럼은 pandas DataFrame의 인덱스로 변환
print(cust_agg.index.name)
cust_agg = cust_agg.reset_index()
cust_agg.head()

cust_country


,cust_country,sum_grade,max_grade
0,GB,8,4
1,KOR,21,7
2,US,12,5


agg() 함수를 활용. agg() 인자로 aggregation 함수 리스트를 입력.  
여전히 aggregation 컬럼별로 API 적용

In [17]:
cust_agg1 = cust_group['grade'].agg(['sum', 'max'])
cust_agg2 = cust_group['age'].agg(['mean', 'max'])
print(cust_agg1.head())
print(cust_agg2.head())

              sum  max
cust_country          
GB              8    4
KOR            21    7
US             12    5
                   mean  max
cust_country                
GB            30.666667   34
KOR           34.250000   67
US            45.500000   77


In [18]:
cust_agg = cust_agg1.merge(cust_agg2, on='cust_country', how='left')
cust_agg.head()

,sum,max_x,mean,max_y
cust_country,,,,
GB,8,4,30.666667,34
KOR,21,7,34.250000,67
US,12,5,45.500000,77


In [19]:
cust_agg = pd.DataFrame()
cust_agg[['sum_grade', 'max_grade']] = cust_group['grade'].agg(['sum', 'max'])
cust_agg[['avg_age', 'max_age']] = cust_group['age'].agg(['mean', 'max'])
cust_agg.head()

,sum_grade,max_grade,avg_age,max_age
cust_country,,,,
GB,8,4,30.666667,34
KOR,21,7,34.250000,67
US,12,5,45.500000,77


agg()에 인자로 Aggregation 컬럼값과 Aggregation함수 리스트를 기재한 dictionary 값 입력

In [35]:
agg_dict = {
    'grade':['sum', 'max'],
    'age':['mean', 'max']
}
cust_agg = cust_group.agg(agg_dict)
cust_agg.head()

grade            age    
               sum max       mean max
cust_country                         
GB               8   4  30.666667  34
KOR             21   7  34.250000  67
US              12   5  45.500000  77

In [22]:
print(cust_agg.columns)
[('_').join(column) for column in cust_agg.columns]

MultiIndex([('grade',  'sum'),
            ('grade',  'max'),
            (  'age', 'mean'),
            (  'age',  'max')],
           )


['grade_sum', 'grade_max', 'age_mean', 'age_max']

groupby 수행 결과 DataFrame의 컬럼명 변경

In [23]:
cust_agg.columns = [('_').join(column) for column in cust_agg.columns]

In [24]:
cust_agg.head()

,grade_sum,grade_max,age_mean,age_max
cust_country,,,,
GB,8,4,30.666667,34
KOR,21,7,34.250000,67
US,12,5,45.500000,77


pandas group by case when  

고객 국가별 grade의 총합과 함께, 국가별로 성별에 따른 grade의 합을 같이 보고자 할때  

- SQL  
SELECT cust_country, sum(grade) total_sum,  
Sum(case when gender=='M' then grade end) male_sum,  
Sum(case when gender=='F' then grade end) female_sum FROM customer GROUP BY cust_country  


case when 조건으로 각각 filtering 하여 별도의 DataFrame 생성

In [25]:
cond_male = customer['gender'] == 'M'
cond_female = customer['gender'] == 'F'
cust_male = customer[cond_male]
cust_female = customer[cond_female]

In [26]:
cust_female

,customer_name,cust_country,gender,grade,age
0,Alice,US,F,3,25
3,Yerin,KOR,F,5,33
5,Katherine,GB,F,3,29
8,Maria,US,F,2,77
10,Jin,KOR,F,5,16


원본 데이터에 cust_country로 groupby 수행

In [27]:
cust_agg = customer.groupby('cust_country')['grade'].sum()
cust_agg = cust_agg.reset_index()

In [28]:
cust_agg.head()

,cust_country,grade
0,GB,8
1,KOR,21
2,US,12


filtering 된 데이터에 cust_country로 groupby 수행

In [29]:
cust_male_agg = cust_male.groupby('cust_country')['grade'].sum()
cust_female_agg = cust_female.groupby('cust_country')['grade'].sum()
cust_male_agg = cust_male_agg.reset_index()
cust_female_agg = cust_female_agg.reset_index()

In [30]:
cust_male_agg

,cust_country,grade
0,GB,5
1,KOR,11
2,US,7


원본 데이터에 groupby 된 데이터 세트와 filtering된 데이터에 groupby된 데이터 세트를 조인

In [31]:
cust_agg = cust_agg.merge(cust_male_agg, on='cust_country', how='left')
cust_agg = cust_agg.merge(cust_female_agg, on='cust_country', how='left')
cust_agg.head()

,cust_country,grade_x,grade_y,grade
0,GB,8,5,3
1,KOR,21,11,10
2,US,12,7,5


In [32]:
cust_agg.columns = ['cust_country', 'sum_grade', 'sum_male_grade', 'sum_female_grade']
cust_agg

,cust_country,sum_grade,sum_male_grade,sum_female_grade
0,GB,8,5,3
1,KOR,21,11,10
2,US,12,7,5
